# Example of hybrid symbolic regression without `pyqg`

In this example, we'll show how to invoke `hybrid_symbolic` on something other than a dataset generated from `pyqg` runs.

## Import dependencies

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import eqn_disco

## Construct the dataset

We'll still need to generate an `xarray.Dataset`, which is going to need to have a specific set of dimensions:
- `x`, zonal (east-west) position in real space
- `y`, meridional (north-south) position in real space
- `k`, zonal position in spectral space (we assume periodic boundary conditions<sup>1</sup>)
- `l`, meridional position in spectral space
- `lev` (optional) - vertical position in real space, ideally only over a small number of values
- `batch` (can have any name) - index of the spatial data sample in the dataset

Data instance shapes must be `(batch, lev, y, x)` or `(batch, y, x)`.

Below is an example of how to initialize such a dataset.


<sup><sup>1</sup>Note that in the future, we hope to extend this library to drop the requirement for periodic boundaries and spectral dimensions, using finite differences to take derivatives instead.</sup>

In [2]:
grid_length = 32
num_samples = 100
num_zlayers = 1

grid = np.linspace(0, 1, grid_length)
inputs = 0.01 * np.random.normal(size=(num_samples, num_zlayers, grid_length, grid_length))

horizontal_wavenumbers = np.arange(0.0, grid_length / 2 + 1)
vertical_wavenumbers = np.append(np.arange(0.0, grid_length / 2), np.arange(-grid_length / 2, 0.0))

data_set = xr.Dataset(
    data_vars={
        "inputs": (("batch", "lev", "y", "x"), inputs),
    },
    coords={
        "lev": np.arange(num_zlayers),
        "x": grid,
        "y": grid,
        "l": vertical_wavenumbers * 2 * np.pi,
        "k": horizontal_wavenumbers * 2 * np.pi,
        "batch": np.arange(num_samples),
    },
)

## Set up the prediction problem

As a demonstration, we're going to set up a problem where the $\mathtt{target}$ expression we're trying to predict consists of two additive terms, one proportional to $\partial_x \nabla^2 [\mathtt{inputs}]$ and the other proportional to $\mathtt{inputs}^2$.

Because of the relative magnitudes of $\mathtt{inputs}$ vs. the grid spacing, these additive terms have extremely different orders of magnitude ($\mathtt{inputs}^2$ is much smaller), so to create an expression that meaningfully includes both of them, we'll need to scale it up quite a bit:

In [3]:
additive_features = ['mul(mul(inputs,inputs),10000000)', 'ddx(laplacian(inputs))']
true_expr = f'add({",".join(additive_features)})'
extractor = eqn_disco.utils.FeatureExtractor(data_set)
data_set['target'] = extractor.extract_feature(true_expr)

for i, feat in enumerate(additive_features):
    print(f"{feat}\n\tstddev = {extractor.extract_feature(feat).std().data}")

mul(mul(inputs,inputs),10000000)
	stddev = 1416.7524144874462
ddx(laplacian(inputs))
	stddev = 5413.163709940415


Although our $\mathtt{target}$ expression is a sum of two relatively simple terms (which you'd think would be easy for any symbolic regression method to learn), there are two difficulties:
1. it contains spatial differential operators, which most symbolic regression libraries can't model
1. it is a very unequally weighted sum, and many symbolic libraries have trouble learning very large or very small constants

Our approach should be able to handle these difficulties.

## Run hybrid symbolic regression

Our approach to symbolic regression is a hybrid between genetic programming and linear regression, interleaved in sequential steps:
1. We run genetic programming to find a symbolic term which is correlated with the target, ignoring the exact proportionality constant.
1. We use linear regression to fit that constant.
1. We subtract our best fit prediction from the target, leaving behind a residual
1. We repeat our approach on the residual, refitting the constants of both terms

Let's try that on this dataset:

In [4]:
terms_by_iter, regressions_by_iter = eqn_disco.hybrid_symbolic.hybrid_symbolic_regression(
    data_set,
    target='target',
    max_iters=2,
    base_features=['inputs'],
    base_functions=['mul'],
    spatial_functions=['ddx', 'ddy', 'laplacian'],
    parsimony_coefficient=0.1
)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.82        0.0635013        3         0.967759         0.964835    116.11m
Iteration 1
Discovered terms: ['ddx(laplacian(inputs))']
Train correlations: [0.9674676]
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.84        0.0133863        7         0.999995         0.999995    115.53m
Iteration 2
Discovered terms: ['ddx(laplacian(inputs))', 'mul(mul(0.096, -1.661), mul(inputs, inputs))']
Train correlations: [1.]


In [6]:
terms_by_iter

['ddx(laplacian(inputs))', 'mul(mul(0.096, -1.661), mul(inputs, inputs))']

From the output, we can see that in the first iteration, we discovered $\partial_x \nabla^2 [\mathtt{inputs}]$, which got us to pretty high (>95%) training correlation, though not all the way to 100%. In the second iteration, we discovered a term corresponding to $\mathtt{inputs}^2$ which got us all the way to 100%, though it included multiplication by a spurious constant (symbolic regression is a bit random and noisy).

Let's look at the coefficients:

In [7]:
regressions_by_iter[-1].models[0].coef_

array([ 1.0000000e+00, -6.2713225e+07])

The first coefficient is exactly 1, and the second becomes 1 if we rescale it to remove the effects of the spurious constants, and divide by the large factor we included earlier:

In [10]:
regressions_by_iter[-1].models[0].coef_[1] * (0.096 * -1.661) / 10000000

0.999999999999998